In [1]:
!mkdir -p frames_1
!ffmpeg -i IMG_7145.MOV -f image2 -q:v 2 -vf "fps=0.2" frames_1/%05d.jpg


Sottodirectory o file -p gi� esistente.
Errore occorso durante l'elaborazione: -p.
ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --e

In [ ]:
import torchvision.transforms as T

train_transforms = T.Compose([
    T.RandomResizedCrop(224, scale=(0.8, 1.0)),  # mild crop
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225])  # typical ImageNet normalization
])

val_transforms = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225])
])


In [24]:
import os
import shutil
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder

# Define the directory containing the images
image_dir = r'C:\Users\barbo\Few Shot VPR\few-shot-vpr\frames_1'

# Create directories for train and test sets with a "frames" subfolder
train_dir = os.path.join(image_dir, 'train', 'frames')  # Add "frames" subfolder
test_dir = os.path.join(image_dir, 'test', 'frames')    # Add "frames" subfolder

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get list of all image files
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

# Split the data into train and test sets
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Move the files to the respective "frames" subfolders
for file in train_files:
    shutil.move(os.path.join(image_dir, file), os.path.join(train_dir, file))

for file in test_files:
    shutil.move(os.path.join(image_dir, file), os.path.join(test_dir, file))

# Create ImageFolder datasets
train_dataset = ImageFolder(root=os.path.join(image_dir, 'train'), transform=train_transforms)
test_dataset = ImageFolder(root=os.path.join(image_dir, 'test'), transform=val_transforms)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [26]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dataset = ImageFolder(root='frames_1/train', transform=train_transforms)
test_dataset   = ImageFolder(root='frames_1/test',   transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader   = DataLoader(test_dataset,   batch_size=8, shuffle=False)


In [38]:
import sys
sys.path.append(r"C:\Users\barbo\Few Shot VPR\few-shot-vpr\cnnimageretrieval-pytorch-master\cnnimageretrieval-pytorch-master")

In [45]:
import torch
import torchvision.transforms as T
from PIL import Image
from cirtorch.networks.imageretrievalnet import init_network

def main():
    # ---------------------------------------------------------
    # 1) Load the pretrained CIRTorch model checkpoint
    # ---------------------------------------------------------
    checkpoint_path = 'retrievalSfM120k-resnet101-gem-b80fb85.pth'  # your .pth file
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print(f">> Loading checkpoint from {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    # The checkpoint has 'state_dict' (weights) and 'meta' (architecture info)
    net_params = checkpoint['meta']  
    net_state_dict = checkpoint['state_dict']

    print(f">> Model architecture: {net_params['architecture']}")
    print(f">> Pooling type: {net_params['pooling']}")

    # Initialize the network
    model = init_network(net_params)     # creates ImageRetrievalNet
    model.load_state_dict(net_state_dict)
    model.eval()  # for inference (switch to train() if you plan to fine-tune)
    model.to(device)

    print(">> Model loaded and moved to device:", device)
    print(">> Output dimension:", model.meta['outputdim'])

    # ---------------------------------------------------------
    # 2) PARTIAL FREEZING (Light Fine-Tuning Setup)
    # ---------------------------------------------------------
    # A) Freeze all parameters first
    for param in model.parameters():
        param.requires_grad = False

    # B) Unfreeze the last ResNet block (layer4) IF it's a ResNet
    if net_params['architecture'].startswith('resnet'):
        # model.features is typically a Sequential of [conv1, bn1, relu, maxpool, layer1, layer2, layer3, layer4]
        # The last element (index -1) is layer4 in ResNet101
        for param in model.features[-1].parameters():
            param.requires_grad = True

    # C) Unfreeze the GeM (or AP-GeM) pooling layer
    for param in model.pool.parameters():
        param.requires_grad = True

    # D) If local/global whitening layers exist, unfreeze them too
    if model.lwhiten is not None:
        for param in model.lwhiten.parameters():
            param.requires_grad = True

    if model.whiten is not None:
        for param in model.whiten.parameters():
            param.requires_grad = True

    # Note: If you plan to do classification, you'd add a small classification head here.
    # E.g. model.classifier = nn.Linear(model.meta['outputdim'], num_classes)

    # If you were going to train, you'd now define an optimizer that only updates
    # the parameters with requires_grad=True. For example:
    #
    # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
    # model.train()

    # ---------------------------------------------------------
    # 3) Define a transform matching model expectations
    # ---------------------------------------------------------
    custom_mean = model.meta['mean']
    custom_std  = model.meta['std']

    test_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=custom_mean, std=custom_std),
    ])

    # ---------------------------------------------------------
    # 4) (Optional) Example: extract a descriptor from 1 image
    # ---------------------------------------------------------
    img_path = 'frames_1/test/frames/00006.jpg'
    img = Image.open(img_path).convert('RGB')
    img_tensor = test_transform(img).unsqueeze(0).to(device)

    # forward pass to extract descriptor
    model.eval()  # ensure eval mode for extraction
    with torch.no_grad():
        descriptor = model(img_tensor)  # shape: [DIM, BATCH]
    descriptor = descriptor.squeeze(1)  # shape [DIM]

    print(">> Descriptor shape:", descriptor.shape)
    print(">> First 5 values:", descriptor[:5])

if __name__ == "__main__":
    main()


>> Loading checkpoint from retrievalSfM120k-resnet101-gem-b80fb85.pth
>> Model architecture: resnet101
>> Pooling type: gem
>> imageretrievalnet.py: for 'resnet101' custom pretrained features 'imagenet-caffe-resnet101-features-10a101d.pth' are used
>> Model loaded and moved to device: cpu
>> Output dimension: 2048
>> Descriptor shape: torch.Size([2048])
>> First 5 values: tensor([4.4055e-02, 1.6614e-08, 1.6614e-08, 1.6614e-08, 1.5044e-02])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torchvision.datasets as datasets

# CIRTorch import
from cirtorch.networks.imageretrievalnet import init_network

###############################
# 1) Define a Classifier Wrapper
###############################
class VPRClassifier(nn.Module):
    """
    Wraps a CIRTorch backbone (which outputs [DIM, BATCH]) with a linear layer 
    for classification (output shape [BATCH, NUM_CLASSES]).
    """
    def __init__(self, backbone, embedding_dim=2048, num_classes=5):
        super().__init__()
        self.backbone = backbone
        self.classifier = nn.Linear(embedding_dim, num_classes) # maybe add 1 more layer in tuning
    def forward(self, x):
        # CIRTorch backbone returns shape [embedding_dim, batch]
        feats = self.backbone(x)  # shape: [DIM, BATCH]
        feats = feats.permute(1, 0)  # reshape to [BATCH, DIM] for classification
        logits = self.classifier(feats)  # [BATCH, num_classes]
        return logits

###############################
# 2) Main Training Script
###############################
# ---------------------------------------------------------
# A) Load the Pretrained CIRTorch Model
# ---------------------------------------------------------
checkpoint_path = "retrievalSfM120k-resnet101-gem-b80fb85.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f">> Loading checkpoint from {checkpoint_path}")
checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

# The checkpoint has 'state_dict' & 'meta'
net_params = checkpoint['meta']       # architecture, pooling, etc.
net_state_dict = checkpoint['state_dict']

print(">> Architecture:", net_params['architecture'])
print(">> Pooling:", net_params['pooling'])

# Init the backbone (CIRTorch ImageRetrievalNet)
backbone = init_network(net_params)
backbone.load_state_dict(net_state_dict)
backbone.eval()  # switch to inference mode for now
backbone.to(device)

print(">> Backbone output dimension:", backbone.meta['outputdim'])

# ---------------------------------------------------------
# B) Partial Freezing (Optional Light Fine-Tuning Setup)
# ---------------------------------------------------------
# 1) Freeze all
for param in backbone.parameters():
    param.requires_grad = False

# 2) Unfreeze only last ResNet block if architecture is ResNet
if net_params['architecture'].startswith('resnet'):
    # The last module in backbone.features is layer4 for resnet101
    for param in backbone.features[-1].parameters():
        param.requires_grad = True

# 3) Unfreeze the pooling layer
for param in backbone.pool.parameters():
    param.requires_grad = True

# 4) (Optional) Unfreeze local/global whitening layers
if backbone.lwhiten is not None:
    for param in backbone.lwhiten.parameters():
        param.requires_grad = True

if backbone.whiten is not None:
    for param in backbone.whiten.parameters():
        param.requires_grad = True

# ---------------------------------------------------------
# C) Create a VPRClassifier (Backbone + Linear Head)
# ---------------------------------------------------------
embedding_dim = backbone.meta['outputdim']  # typically 2048 for ResNet101
num_classes = 5  # e.g., 5 buildings
model = VPRClassifier(backbone, embedding_dim, num_classes)
model = model.to(device)

# ---------------------------------------------------------
# D) Define Dataset, Transforms, and Dataloaders
# ---------------------------------------------------------
# Example using standard ImageFolder structure:
#   data/train/class_0/*.jpg
#   data/train/class_1/*.jpg
#   ...
#   data/val/class_0/*.jpg
#   data/val/class_1/*.jpg
# etc.

# Use the mean/std from the backbone metadata
train_transforms = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean=backbone.meta['mean'], std=backbone.meta['std']),
])

val_transforms = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=backbone.meta['mean'], std=backbone.meta['std']),
])

# Replace 'path_to_train' and 'path_to_val' with your actual folders
train_dataset = ImageFolder(root='frames_1/train', transform=train_transforms)
test_dataset   = ImageFolder(root='frames_1/test',   transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(test_dataset,   batch_size=8, shuffle=False, num_workers=2)

# ---------------------------------------------------------
# E) Define Optimizer, Loss, and Move Model to Train Mode
# ---------------------------------------------------------
criterion = nn.CrossEntropyLoss() # Contrastive loss here
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=1e-4, 
    weight_decay=1e-5
)

model.train()  # enable train mode for dropout, BN, etc.

# ---------------------------------------------------------
# F) Training Loop
# ---------------------------------------------------------
num_epochs = 5  # or whatever you like (add early stopping)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # forward pass
        logits = model(images)  # [batch_size, num_classes]
        loss = criterion(logits, labels)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # accumulate stats
        running_loss += loss.item() * images.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    # Validate
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_logits = model(val_images)
            v_loss = criterion(val_logits, val_labels)

            val_loss += v_loss.item() * val_images.size(0)
            val_preds = torch.argmax(val_logits, dim=1)
            val_correct += torch.sum(val_preds == val_labels).item()
            val_total += val_labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}], "
            f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print(">> Training complete!")




>> Loading checkpoint from retrievalSfM120k-resnet101-gem-b80fb85.pth
>> Architecture: resnet101
>> Pooling: gem
>> imageretrievalnet.py: for 'resnet101' custom pretrained features 'imagenet-caffe-resnet101-features-10a101d.pth' are used
>> Backbone output dimension: 2048
Epoch [1/5], Train Loss: 1.5632, Train Acc: 0.7963, Val Loss: 1.5010, Val Acc: 1.0000
Epoch [2/5], Train Loss: 1.4812, Train Acc: 1.0000, Val Loss: 1.4007, Val Acc: 1.0000
Epoch [3/5], Train Loss: 1.4030, Train Acc: 1.0000, Val Loss: 1.3198, Val Acc: 1.0000
Epoch [4/5], Train Loss: 1.3364, Train Acc: 1.0000, Val Loss: 1.2706, Val Acc: 1.0000
Epoch [5/5], Train Loss: 1.2709, Train Acc: 1.0000, Val Loss: 1.2363, Val Acc: 1.0000
>> Training complete!


In [52]:
import torch
import torchvision.transforms as T
from PIL import Image
from cirtorch.networks.imageretrievalnet import init_network

###########################
# Suppose you trained like this:
###########################

# 1) We assume 'model' is already trained and in memory, or re-load it:
# model = VPRClassifier(...)
# model.load_state_dict(...)
# model.eval()

# or you can just define a placeholder if you want to test structure:
# model = ...
# device = ...

# 2) Define your transform (same as in training)
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    # use the same normalization as training
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

# 3) Suppose you have 5 classes
class_names = ['buildingA', 'buildingB', 'buildingC', 'buildingD', 'buildingE']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###########################
# test_single_image function
###########################
def test_single_image(model, img_path, transform, device, class_names):
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        logits = model(img_tensor)
        preds = torch.argmax(logits, dim=1)

    pred_class_idx = preds.item()
    pred_class_name = class_names[pred_class_idx]

    print("Predicted class ID:", pred_class_idx)
    print("Predicted class name:", pred_class_name)

    return pred_class_idx, pred_class_name

###########################
# Now do the test
###########################
img_path = 'frames_1/test/frames/00006.jpg'
test_single_image(model, img_path, transform, device, class_names)


Predicted class ID: 0
Predicted class name: buildingA


(0, 'buildingA')

In [ ]:
import os

def process_video(video_path, frame_rate, output_folder='dataset'):
    """
    Processes a video, extracts frames at a specified frame rate, and saves them to a subdirectory.
    Args:
        video_path: Path to the input video file.
        frame_rate: Desired frame rate for extraction.
        output_folder: The name of the folder to store the frames
    """

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    subfolder_name = os.path.join(output_folder, video_name)  
    os.makedirs(subfolder_name, exist_ok=True)

    # Remove the "!" from the command so it runs correctly with os.system
    ffmpeg_command = f"ffmpeg -i '{video_path}' -f image2 -q:v 2 -vf 'fps={frame_rate}' '{subfolder_name}/%05d.jpg'"
    print(f"Executing: {ffmpeg_command}")
    os.system(ffmpeg_command)

# Example usage (in Colab):
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/CV videos')

process_video('IMG_7145.MOV', 0.2)
!ls -l dataset
